In [1]:
import torch

In [2]:
def fill_zero_places(a):
    a = torch.cat([torch.tensor([-1], device=a.device), a])
    nonzero_indices = torch.nonzero(a).squeeze()
    diff = torch.diff(torch.cat([nonzero_indices, torch.tensor([a.shape[0]], device=a.device)]))
    b = a[nonzero_indices].repeat_interleave(diff)
    b[b < 0] = 0
    c = b[1:]
    return c

In [3]:
def shift_values_to_next_available_zero(a):
    b = (a > 0).int()
    b = torch.cat([torch.tensor([0], device=a.device), b])
    c = torch.arange(b.shape[0], device=a.device) * b
    c[-1] = 0
    d = c[1:]
    e = torch.roll(fill_zero_places(d), 1)
    e[0] = 0
    f = torch.zeros_like(a)
    f.scatter_add_(0, e.long(), a)
    return f

In [4]:
def fill_zero_values_ascending_rest_with_zero(a):
    b = (a > 0).int()
    cumsum = torch.cumsum(1 - b, dim=0)
    c = (cumsum * (1 - b) - torch.maximum(b * cumsum, torch.zeros_like(cumsum)).cummax(dim=0)[0]) * (1 - b)
    return c

In [5]:
def find_one_zero_pattern(a):
    b = torch.roll(a.int(), -1)
    b[-1] = 1
    c = a - b
    d = (c == 1).int()
    return d

In [6]:
def fill_zero_values_descending_rest_with_zero(a):
    b = fill_zero_values_ascending_rest_with_zero(a)
    c = (b > 0).int()
    d = find_one_zero_pattern(c)
    e = d * b
    f = shift_values_to_next_available_zero(e)
    g = fill_zero_places(f)
    h = ((g + 1) - b) * c
    return h

In [52]:
def find_values_that_must_be_reordered(a):
    b = torch.arange(1, a.shape[0] + 1)
    c = a - b
    d, _ = torch.cummin(c.flip(0), dim=0)
    d = d.flip(0)
    e = (c == d).int()

    return 1 - e

In [75]:
def adjust_cars_no_lane_change(a):
    b = 1 - find_values_that_must_be_reordered(a)
    c = fill_zero_values_descending_rest_with_zero(b)
    d = a * b
    e = shift_values_to_next_available_zero(d)
    f = fill_zero_places(e)
    g = f - c

    return g

In [54]:
a = torch.tensor([ 83,  84,  89,  93,  87,  92,  90,  95,  90,  94,  94,  96,  97, 101,  97, 105, 104,  99, 101, 101, 101, 110, 105, 111, 106, 115, 90,  86, 107,  59])
adjust_cars_no_lane_change(a)
# find_values_that_must_be_reordered(a).int()

tensor([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
        48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [76]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
adjust_cars_no_lane_change(a)

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36])

In [73]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
# c = torch.where((torch.roll(m, 1) == 1) & (m == 0), torch.tensor(1, device=a.device), torch.tensor(0, device=a.device))
b = 1 - find_values_that_must_be_reordered(a)
c = fill_zero_values_descending_rest_with_zero(b)
d = a * b
e = shift_values_to_next_available_zero(d)
f = fill_zero_places(e)
g = f - c
# d = (torch.cumsum(1 - c, dim=0) * (1 - c) - torch.maximum(c * torch.cumsum(1 - c, dim=0), torch.zeros_like(c)).cummax(dim=0)[0]) * (1 - c)
# e = a * c
# nonzero_indices = torch.nonzero(e, as_tuple=True)[0]
# repeat_counts = torch.diff(torch.cat((nonzero_indices, torch.tensor([e.shape[0]]))))
# f = e[nonzero_indices].repeat_interleave(repeat_counts)
# g = torch.roll(torch.argsort(e).cummax(dim=0)[0] * c, 1)
# h = torch.zeros_like(e)
# h.scatter_(0, g.long(), e)
# i = h.cummax(dim=0)[0]
# j = i - d
# l = i - k
# n = 1 - m
# o = fill_zero_values_descending_rest_with_zero(n) * m
# p = (i - o) * m
# q = p + n * a

# a, m, c, d, e, f, g, h, i, j, k

a, b, c, d, e, f, g

(tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24,
         23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36]),
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 0, 0, 1, 1], dtype=torch.int32),
 tensor([4, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 8, 7, 6, 5, 4, 3, 2, 1, 0, 1, 0,
         4, 3, 2, 1, 0, 0]),
 tensor([ 0,  0,  0,  0,  5,  0,  7,  0,  0,  0,  0,  0, 13,  0,  0,  0,  0,  0,
          0,  0,  0, 22,  0, 26,  0,  0,  0,  0, 31, 36]),
 tensor([ 5,  0,  0,  0,  0,  7,  0, 13,  0,  0,  0,  0,  0, 22,  0,  0,  0,  0,
          0,  0,  0,  0, 26,  0, 31,  0,  0,  0,  0, 36]),
 tensor([ 5,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 22, 22, 22, 22, 22,
         22, 22, 22, 22, 26, 26, 31, 31, 31, 31, 31, 36]),
 tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 25, 26, 27, 28, 29, 30, 31, 36]))

In [32]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 29, 32, 31, 36])
b = torch.arange(1, a.shape[0] + 1)
c = a - b
d, _ = torch.cummin(c.flip(0), dim=0)
d = d.flip(0)
e = (c == d).int()

# a, b, c, d
c, d, e

(tensor([2, 8, 5, 7, 0, 7, 0, 6, 9, 5, 8, 8, 0, 7, 1, 8, 2, 6, 4, 2, 7, 0, 4, 2,
         8, 6, 2, 4, 2, 6]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
         2, 2, 2, 2, 2, 6]),
 tensor([0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 1, 1], dtype=torch.int32))

In [11]:
a = torch.tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24, 23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36])
b, _ = torch.cummin(a.flip(0), dim=0)
b = b.flip(0)
c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
d = a * c
e = ((d >= b).int() * c).int()
f = (1 - c)
g = fill_zero_values_descending_rest_with_zero(f)
h = d - g
i = ((h > b).int() * c | (a >= b).int() * c).int()

a, b, c, d, e, f, g, h, i, a  * i, a  * (1-i)

(tensor([ 3, 10,  8, 11,  5, 13,  7, 14, 18, 15, 19, 20, 13, 21, 16, 24, 19, 24,
         23, 22, 28, 22, 27, 26, 33, 32, 30, 32, 31, 36]),
 tensor([ 3,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 16, 16, 19, 19, 22,
         22, 22, 22, 22, 26, 26, 30, 30, 30, 31, 31, 36]),
 tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
         1, 1, 0, 1, 0, 0]),
 tensor([ 0, 10,  8, 11,  0, 13,  0, 14, 18, 15, 19, 20,  0, 21,  0, 24,  0, 24,
         23, 22, 28,  0, 27,  0, 33, 32,  0, 32,  0,  0]),
 tensor([0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
         1, 1, 0, 1, 0, 0], dtype=torch.int32),
 tensor([1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
         0, 0, 1, 0, 1, 1]),
 tensor([0, 3, 2, 1, 0, 1, 0, 5, 4, 3, 2, 1, 0, 1, 0, 1, 0, 4, 3, 2, 1, 0, 1, 0,
         2, 1, 0, 1, 0, 0]),
 tensor([ 0,  7,  6, 10,  0, 12,  0,  9, 14, 12, 17, 19,  0, 20,  0, 23,  0, 20,
         20, 20, 27,  0, 26,  0, 31, 31,

In [12]:
tensor([ 3,  5,  5,  5,  5,  7,  7, 13, 13, 13, 13, 13, 13, 16, 16, 19, 19, 22, 22, 22, 22, 22, 26, 26, 30, 30, 30, 31, 31, 36])

NameError: name 'tensor' is not defined

In [ ]:
a = torch.tensor([11, 12, 23, 14, 14, 13, 60, 60])
b, _ = torch.cummin(a.flip(0), dim=0)
b = b.flip(0)
c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
d = a * c
e = ((d >= b).int() * c).int()
f = (1 - c)
g = fill_zero_values_descending_rest_with_zero(f)
h = d - g
i = ((h > b).int() * c | (a >= b).int() * c).int()

a, b, c, d, e, f, g, h, i

In [ ]:
def find_values_to_reorder_from_argmin(a):
    """
    a = array([3, 10, 10, 11, 11, 11, 12, 12, 12, 13])

    return = array([3, 5, 6, 7, 8, 9, 10, 11, 12, 13])
    """

    pass

In [ ]:
b = torch.tensor([3, 10, 10, 11, 11, 11, 12, 12, 12, 13])
c = torch.cat([(1 - (b[1:] - b[:-1]) > 0).int(), torch.tensor([0], device=a.device)])
d = a * c
e = ((d >= b).int() * c).int()
f = (1 - c)
q = a * torch.arange(1, a.shape[0] + 1)
p = b * f
r = p * torch.arange(1, b.shape[0] + 1)

b, c, d, e, q, p, r

In [ ]:
a = torch.tensor([11, 12, 23, 14, 14, 13, 60, 60])
b, _ = torch.cummin(a.flip(0), dim=0)

a, b

In [ ]:
a = torch.tensor([ 1,  4,  3,  6,  5,  6,  8,  8, 14, 10, 11, 13, 13, 14, 15, 16, 17, 18,  19, 20, 21, 23, 23, 24, 25, 26, 27, 28, 30, 35])
# find_values_that_must_be_reordered(a).int()
adjust_cars_no_lane_change(a)

In [ ]:
count = 30000
lane_num_tracks = torch.tensor([300000])
cars_position = torch.arange(0, count)
cars_speed = torch.randint(0,10, (count, 1)).flatten()

adjust_cars_no_lane_change(cars_position + cars_speed)

### Benchmarks

In [ ]:
# import torch
# import time

# # Ensure CUDA is available
# if not torch.cuda.is_available():
#     raise RuntimeError("CUDA is not available, please run on a CUDA-capable device.")

# # Number of iterations
# iterations = 1000000
# count = 3000000

# lane_num_tracks = torch.tensor([300000000]).cuda()
# cars_position = torch.arange(0, count).cuda()
# cars_speed = torch.randint(0, 10, (count, 1)).flatten().cuda()

# def adjust_cars_no_lane_change(cars_info):
#     # Sample implementation, replace with the actual function
#     return cars_info * 2  # Example operation

# # Start timing
# start_time = time.time()

# # Execute the function multiple times
# for _ in range(iterations):
#     result = adjust_cars_no_lane_change(cars_position + cars_speed)

# # Optionally, synchronize and free the cache to get accurate timings
# torch.cuda.synchronize()
# torch.cuda.empty_cache()

# # End timing
# end_time = time.time()

# # Calculate elapsed time
# elapsed_time = end_time - start_time
# print(f"Elapsed time: {elapsed_time} seconds")

Iterationen: 1.000.000
Autos: 300.000

Zeit: 16 Sekunden

Iterationen: 10.000.000
Autos: 30.000

Zeit: 94 Sekunden

Iterationen: 1.000.000
Autos: 3.000.000

Zeit: 178 Sekunden